# Projekt 5 Markus Kankkonen


### Uppgift 1
- Vi börjar med att skapa nödvändiga funktioner.
- showImage visar en bild med optional text och textcolor parametrar
- cornerHarris kör bilden genom cv2s cornerHarris funktion

In [10]:
import cv2 
import numpy as np
import copy

def showImage(image,text="",text_color=(0,0,250)):
    text_img = copy.deepcopy(image)
    text_img = cv2.putText(text_img,text,(20,40),cv2.FONT_HERSHEY_SIMPLEX,1,text_color,2)
    cv2.namedWindow("main", cv2.WINDOW_NORMAL)
    #cv2.resizeWindow('image', 900, 450) 
    cv2.imshow("main",text_img)
    cv2.waitKey(0)
    cv2.waitKey(0)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def cornerHarris(image,thresh=200,blockSize=2,ksize=3,k=0.04):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # cornerharris funktionen
    img_ch = cv2.cornerHarris(gray,blockSize,ksize,k)
    # Vi skapar en tom lista av samma shape som vårt resultat från cornerharris
    img_ch_norm = np.zeros(img_ch.shape, dtype=np.float32)

    # Vi Normaliserar våra värden mellan 0,255
    img_ch_norm = cv2.normalize(img_ch, img_ch_norm, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # convertScaleAbs skalar våran lista och converterar till 8-bit int
    img_ch_norm = cv2.convertScaleAbs(img_ch_norm)
    # vi gör en kopia på listan som vi sedan ritar in våra cirklar i 
    result_ch = copy.deepcopy(img_ch_norm)
    result = copy.deepcopy(image)
    # Här söker vi efter värden på våran bild som är större än våran threshold och där ritar vi en cirkel.
    for i in range(img_ch_norm.shape[0]):
        for j in range(img_ch_norm.shape[1]):
            if int(img_ch_norm[i,j]) > thresh:
                cv2.circle(result, (j,i), 5, (0,0,255), 2)
                cv2.circle(result_ch, (j,i), 5, (0,0,255), 2)

    #showImage(img_ch_norm_scaled)
    return result, result_ch

- Vi läser in våran bild

In [28]:
image = cv2.imread("house.jpg")
res1, res2 = cornerHarris(image,101,6,5,0.0561)
joined_img = np.hstack((image,res1))
showImage(joined_img)

- Det visade sig vara väldigt svårt att få en perfekt bild enligt min åsikt. Men problemet kan vara att jag inte direkt vet vad jag söker efter utan bara testar mig fram
- Jag får sådana bilder som har alltför mycket prickar eller så sådana som har helt ändast de största edges som finns i bilden, oftast om den hittar en corner så blir känner den igen flera corners direkt i närheten så cirklarna blir klumppade ihop.
- Jag har inkluderat 3 exempel på samma bild en mera våldsam corner detection, och en mera konservativ och en mitt emellan :)



## Easy

![result corner harris](house_result.png)


## Medium

![house result corner](house_result_2.png)

## Extreme

![house result corner](house_result_3.png)

### Uppgift 2
- Vi använder cv2.ORB för att hitta matchande egenskaper.

In [101]:
# Vi läser in våra bilder
img1 = cv2.imread("cat.jpg",0)
img2 = cv2.imread("match.jpeg",0)

In [102]:
# Vi skapar en Orb instans
orb = cv2.ORB_create()
# Vi söker efter så säga keypoints och descriptors i bilderna 
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

In [103]:
# Matchern försöker slå ihop matchande keypoints i båda bilderna.
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(des1,des2) 
# Sortera bilderna enligt distance desto kortare desto bättre.
matches = sorted(matches, key = lambda x:x.distance,reverse=False)
# Rita ut våra matches och spara den nya bilden tillen array.
img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

showImage(img3)

## De 30 resultat som har kortaste distance 

![resultat](matching_result_2.png)

## De 30 resultat som har längst distance 

![resultat](matching_result.png)

## Alla resultat

![resultat](matching_result_3.png)

- Som man ser från bilderna så var de 30 matches med längst distans alla på rätt objekt. 
- Detta tycket jag mig hända på flera av de bilder jag testade. 
- Det stora problemet känns det som att ORBen bara tar så små egenskaper som den sen matchar med vad som helst. Om man börjar zooma in på bilden så märker man att många sträck inte går till exakt rätt punkt, dvs. flera punkter på nån bokstav från tändsticks paketet matchas med en annan punkt med liknande features på en annan bokstav


### Uppgift 3

- Vi gör ett program som kan hitta och skilja på trianglar, rektanglar, pentagoner, hexagoner och septagoner


In [63]:
def findShapes(contours,image):
    print(len(contours))
    for i,elem in enumerate(contours):
        # räkn ut omkrets för våran symbol
        perimeter = cv2.arcLength(elem,True)
        # epsilon värde, Lekte runt en stund med denna parameter också 0.05 verkade bra. 
        epsilon = 0.05*perimeter
        # räkna ut var punkterna ligger
        approxC = cv2.approxPolyDP(elem,epsilon,True)
        text = ""
        # Ta x, y för den punk som ligger högst uppe, längst till vänster
        # Samt bredd och höjd för hela figuren
        x,y,w,h = cv2.boundingRect(elem)
        # Beroende på hur många punkter symbolen har ange den ett namn.
        if (len(approxC) == 3):
            text = "triangle"
        elif (len(approxC) == 4):
            # hitta skillnader mellan kvadrat och rektangel
            if (w / h <= 1.1 and w / h >= 0.9):
                text = "square"
            else: 
                text = "rectangle" 
        elif (len(approxC) == 5):
            text = "pentagon"
        elif (len(approxC) == 6):
            text = "hexagon"
        elif (len(approxC) == 7):
            text = "septagon"   
        else:
            text = str(len(approxC))
        # Sätt in texten på bilden.  
        cv2.putText(image,text,(x-20,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,0),2)




In [64]:
image = cv2.imread("shape2.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

img_shape = copy.deepcopy(image)
# Vi tar thresholden på vår bild
ret, thresh = cv2.threshold(image, 240, 255, cv2.THRESH_BINARY)
# Vi använder Canny för att bättre få fram konturerna utan hittar den inte symbolerna lika lätt
filtered = cv2.Canny(thresh,200,300,3)
# findCountours söker efter konturerna för oss
contours, _ = cv2.findContours(filtered, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

findShapes(contours,image)


# sett våran input bild samt ouputten från våran algoritm brevid varandra.
joined_img = np.hstack((img_shape,cv2.drawContours(image, contours, -1, (0,255,255), 2)))
showImage(joined_img)

11


### Input till vänster, output till höger
![resultat](shape_result.png)

- Det tog en stund att förstå hur man skulle sätta ihop allt men överlag ett väldigt intressant projekt
- Med canny började jag få bättre resultat. 
- Yrade dock runt en bra stund med olika parametrar hit och dit
- Vissa av symbolerna blir duplikat av någon orsak. Längden på min contours lista blir 11 så alla utom 1 blir duplikat. Outputten endå samma fasten vissa skrivs flera gånger.
